# Finding Near real-time current data

In [1]:
import os
import sys
import time
import warnings

ioos_tools = os.path.join(os.path.pardir)
sys.path.append(ioos_tools)

In [2]:
from datetime import datetime, timedelta

# Region.
bbox = [-123, 36, -121, 40]
crs = 'urn:ogc:def:crs:OGC:1.3:CRS84'
    
# Temporal range.
now = datetime.utcnow()
start,  stop = now - timedelta(days=(7)), now

# Names.
cf_names = ['*sea_water_potential_temperature*',
            '*sea_water_salinity*']

In [6]:
from owslib import fes
from ioos_tools.ioos import fes_date_filter

kw = dict(wildCard='*', escapeChar='\\',
          singleChar='?', propertyname='apiso:AnyText')

or_filt = fes.Or([fes.PropertyIsLike(literal=('*%s*' % val), **kw)
                  for val in cf_names])

# Exclude ROMS Averages and History files.
not_filt = fes.Not([fes.PropertyIsLike(literal='*GNOME*', **kw)])

services = ['OPeNDAP','SOS'] 
#services = ['asdfasfasdf','asfasdfasdfasd']
service_filt = fes.Or([fes.PropertyIsLike(propertyname='apiso:ServiceType',literal=('*%s*' % val),
                        escapeChar='\\',wildCard='*',singleChar='?') for val in services])

begin, end = fes_date_filter(start, stop)
bbox_crs = fes.BBox(bbox, crs=crs)
filter_list = [fes.And([bbox_crs, begin, end, or_filt, not_filt])]
filter_list = [fes.And([or_filt, not_filt])]
filter_list = [fes.And([or_filt, not_filt, service_filt])]

In [7]:
from owslib.csw import CatalogueServiceWeb


catalogs = ['http://www.ngdc.noaa.gov/geoportal/csw',
            'https://dev-catalog.ioos.us/csw',
            'http://geoport.whoi.edu/csw',
            'http://catalog.data.gov/csw-all']

catalogs = ['https://dev-catalog.ioos.us/csw',
            'http://geoport.whoi.edu/csw',
            'http://catalog.data.gov/csw-all']

for endpoint in catalogs:
    csw = CatalogueServiceWeb(endpoint, timeout=60)
    csw.getrecords2(constraints=filter_list, maxrecords=1000, esn='full')
    print(endpoint)
    print(len(csw.records))
    print(csw.records.keys())

https://dev-catalog.ioos.us/csw
10
['deployments/mbari/UCSC294-20150430T2218/UCSC294-20150430T2218.nc3.nc', 'ud_134-20150122T1955', 'ru24-20150105T1441', 'NS15agg', 'deployments/lfiorentino/gichigami-20110706T1754/gichigami-20110706T1754.nc3.nc', 'edu.maine:A01', 'gov.noaa.nos.ioos:nanoos_regional_data_portal', 'gov.noaa.nos.ioos:caricoos_regional_data_portal', 'gov.noaa.nos.ioos:aoos_regional_data_portal', 'sp025-20140611T184300']
http://geoport.whoi.edu/csw
10
['USGS_BBLEH_SANDY_076', 'COAWST.Barnegat_Bay.Spring2012', 'gov.usgs.cmg:USGS_CHINCO_SANDY', 'gov.usgs.cmg:USGS_CHINCO_EPR', 'gov.usgs.cmgp:COAWST.USEAST.Forecast', 'gov.usgs.cmg:HudJBay_Sandy', 'COAWST.MVCO.CBLAST.spatial_7_ar0fd', 'gom3_nocache', 'massbay_nocache', 'roms_hiig_assimilation']
http://catalog.data.gov/csw-all
10
['35cce7bd-783b-45d6-9e1d-020f3a2ed95c', '0e9f6bbd-c5f8-454d-8309-1437c27ed6dd', '506d7521-cc62-4e72-a5b6-f9dc578db637', '9d09db91-69e2-426c-b6e3-03e726453f9b', 'gov.noaa.nodc:0084099', 'maui_water_qualit